<h1 style="text-align:center;font-size:36px">NOTEBOOK DE BAC A SABLE</h1>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
from shapely.geometry import Point
from shapely.ops import transform
import pyproj
import re, os
from datetime import time, datetime
import Connexion_Transfert as ct
import Import_trafics as it
import Donnees_horaires as dh
import Donnees_sources as ds
import Outils as O
from Params import Bdd_OTV as pb

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 500)

# donnees Tipi

In [ ]:
donneesMinutes = pd.read_csv(r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', delimiter=',')

In [ ]:
# liste des stations TIPI
listIdCompteurTipi=donneesMinutes.PME_ID.unique()
# liste des stations OTV
with ct.ConnexionBdd('local_otv_boulot') as c: 
    listObsSupl  =pd.read_sql("select distinct id_comptag, id_cpt, obs_supl from comptage.compteur where gestionnai='DIRA'", c.sqlAlchemyConn)
listStationOtvBrut = listObsSupl.obs_supl.apply(lambda x : x.split(';')[1].split('station : ')[1] if not pd.isnull(x) and 'EMC' not in x and ';' in x else x).unique()
listStationOtvBrut2 = [e.split(',') for e in listStationOtvBrut if e and re.search('^M.*\.._.*$', e)]
listStationOtv = [a.strip().replace('_','') for b in listStationOtvBrut2 for a in b]

In [ ]:
# station TIPI non présentes dans OTV : 
StationTipiHorsOTV = [s for s in listIdCompteurTipi if s not in listStationOtv]
StationOtvHorsTipi = [s for s in listStationOtv if s not in listIdCompteurTipi]
StationOtvTipi = [s for s in listIdCompteurTipi if s in listStationOtv]

# TEST OUVERTUR FICHIERE TAGMASTER .DAT

# donnees CD16 comptages temporaires

In [ ]:
cd16Temp=ds.FIM(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires\14-5.fim', gest='CD16')

In [ ]:
cd16Temp.geoloc, cd16Temp.geom_l93.wkt

In [ ]:
geom_l93

In [ ]:
# parcourir un dossier, vérifier si le le FIM est geoloc et récupérer la geom si c'est le cas
dossierCptTemp=r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires'
dicoDf={'fichier': [], 'geom': [], 'tmja': [], 'pc_pl': []}
with os.scandir(dossierCptTemp) as it:
    for e in it:
        if e.is_file() and e.name.lower().endswith('.fim'):
            cpt=ds.FIM(e, verifQualite='Message')
            if cpt.geoloc:
                dicoDf['fichier'].append(e.name)
                dicoDf['geom'].append(cpt.geom_l93)
                dicoDf['tmja'].append(cpt.tmja)
                dicoDf['pc_pl'].append(cpt.tmja)

In [ ]:
df=pd.DataFrame(dicoDf)
gdf=gp.GeoDataFrame(df, geometry=df.geom, crs='epsg:2154')
gdf.drop('geom', axis=1).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\testGeomTemp.shp')

In [ ]:
cpt.geom_l93.wkt

In [ ]:
wgs84Proj = pyproj.CRS('EPSG:4326')
l93Proj = pyproj.CRS('EPSG:2154')
project = pyproj.Transformer.from_crs(wgs84Proj, l93Proj).transform
geoloc=re.search('(?P<lat>(\-|\+)[0-9]{1,3}\.([0-9]{4}\.){2})(?P<long>(\-|\+)[0-9]{1,3}\.([0-9]{4}\.){2})', '+45.6021.6666.-000.2134.6333.')
longitude = geoloc.group('long')
latitude = geoloc.group('lat')
x_wgs84=float(re.sub('(\+|-)(.*\.)(.*)(\.)(.*)(\.)', '\g<1>\g<2>\g<3>\g<5>', longitude))
y_wgs84=float(re.sub('(\+|-)(.*\.)(.*)(\.)(.*)(\.)', '\g<1>\g<2>\g<3>\g<5>', latitude))
geom_wgs84=Point(y_wgs84, x_wgs84 )
geom_l93 = transform(project, geom_wgs84)

In [ ]:
geom_l93.wkt

In [ ]:
cpt.dfHoraire2Sens.pivot(index=['date', 'type_veh', 'fichier'], columns='heure', values='nbVeh').reset_index().rename(
    columns={k: v for k, v in zip(['date', 'type_veh']+[e for e in range(24)], ['jour', 'indicateur']+dh.attributsHoraire)})

In [ ]:
cpt.dfHoraire2Sens

# test donnees 6 minutes vikings

In [125]:
listeResultats=[]

In [205]:
vik = ds.Viking(r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\pont_GeorgeTriaud\201_voie_g\total.VIK')

0002. 017.0001.  90.   1.  21.  12.  13.  13.  00.0300.1200.   1. +46.1812.6332.-001.1393.9499.


In [206]:
# correction du SER
test = vik.dfFichier.copy()
def corrTypeVeh(ser, typeVehExistant):
    if typeVehExistant == '2R':
        return typeVehExistant
    elif int(ser.split('S=')[1]) >= 2112:
        return 'PL'
    else:
        return 'VL'
test['type_veh'] = test.apply(lambda x : corrTypeVeh(x.ser, x.type_veh), axis=1)

In [207]:
dfGroupe = ds.GroupeCompletude(ds.NettoyageTemps(test), '6T', vitesse=True)#.iloc[:100]

['sens1']


In [208]:
dfGroupe.pivot(index=['date_heure'], columns=['type_veh'], values=['nbVeh', 'v50']).reset_index().assign(voie='Voie gauche', capteur='Pont George Triaud').to_csv(
r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\pont_GeorgeTriaud\201_voie_g\Pont George Triaud_voie_g.csv')

In [209]:
listeResultats.append(dfGroupe.pivot(index=['date_heure'], columns=['type_veh'], values=['nbVeh', 'v50']).reset_index().assign(
    voie='Voie gauche', capteur='Pont George Triaud'))

In [210]:
listeResultats[3]

date_heure nbVeh                v50                    \
type_veh                        2R   PL   TV   VL  2R  PL     TV     VL   
0        2021-12-14 00:06:00   0.0  0.0  2.0  2.0 NaN NaN  131.0  131.0   
1        2021-12-14 00:12:00   0.0  0.0  0.0  0.0 NaN NaN    NaN    NaN   
2        2021-12-14 00:18:00   0.0  0.0  1.0  1.0 NaN NaN  118.0  118.0   
3        2021-12-14 00:24:00   0.0  0.0  0.0  0.0 NaN NaN    NaN    NaN   
4        2021-12-14 00:30:00   0.0  0.0  0.0  0.0 NaN NaN    NaN    NaN   
...                      ...   ...  ...  ...  ...  ..  ..    ...    ...   
8394     2022-01-17 23:30:00   0.0  0.0  1.0  1.0 NaN NaN  123.0  123.0   
8395     2022-01-17 23:36:00   0.0  0.0  0.0  0.0 NaN NaN    NaN    NaN   
8396     2022-01-17 23:42:00   0.0  0.0  0.0  0.0 NaN NaN    NaN    NaN   
8397     2022-01-17 23:48:00   0.0  0.0  1.0  1.0 NaN NaN  101.0  101.0   
8398     2022-01-17 23:54:00   0.0  0.0  1.0  1.0 NaN NaN  115.0  115.0   

                 voie             capteur  
type_veh                                   
0         Voie gauche  Pont George Triaud  
1         Voie gauche  Pont George Triaud  
2         Voie gauche  Pont George Triaud  
3         Voie gauche  Pont George Triaud  
4         Voie gauche  Pont George Triaud  
...               ...                 ...  
8394      Voie gauche  Pont George Triaud  
8395      Voie gauche  Pont George Triaud  
8396      Voie gauche  Pont George Triaud  
8397      Voie gauche  Pont George Triaud  
8398      Voie gauche  Pont George Triaud  

[8399 rows x 11 columns]

In [211]:
pd.concat(listeResultats).to_csv(r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\concatenation6Minutes.csv')

In [ ]:
vik.dfFichier

# Donnees Vitesse Bordeaux ville zone 30

In [ ]:
dfGroupe.to_csv(r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\P5_.csv')

In [ ]:
dfValide = ds.NettoyageTemps(vik.dfFichier)
frequence='6T'

In [ ]:
dfGroupTypeHeure=dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'type_veh','sens'])['nbVeh'].count().reset_index().sort_values('date_heure')
#completude des données
#ajout des données horaires à 0 si aucun type de vehicules mesures
date_range=pd.date_range(dfGroupTypeHeure.date_heure.min(),dfGroupTypeHeure.date_heure.max(),freq=frequence)
# df de comparaison
dfComp=pd.DataFrame({'type_veh':['2R','PL','VL']}).assign(key=1).merge(pd.DataFrame({'date_heure':date_range}).assign(key=1), on='key').merge(
pd.DataFrame({'sens':['sens1','sens2']}).assign(key=1)).sort_values(['date_heure', 'type_veh','sens'])[['date_heure','type_veh','sens']]
# df des données amnquantes
dfManq=dfComp.loc[dfComp.apply(lambda x : (x['date_heure'],x['type_veh'],x['sens']) not in zip(dfGroupTypeHeure.date_heure.tolist(),dfGroupTypeHeure.type_veh.tolist(),
                  dfGroupTypeHeure.sens.tolist()), axis=1)].copy()
dfHeureTypeSens = pd.concat([dfManq,dfGroupTypeHeure],axis=0).fillna(0).sort_values(['date_heure', 'type_veh','sens'])
dfHeureTypeSens['jour']=dfHeureTypeSens.date_heure.dt.dayofweek
dfHeureTypeSens['jourAnnee']=dfHeureTypeSens.date_heure.dt.dayofyear
dfHeureTypeSens['heure']=dfHeureTypeSens.date_heure.dt.hour
dfHeureTypeSens['date']=pd.to_datetime(dfHeureTypeSens.date_heure.dt.date)
dfHeureTypeSens=pd.concat([dfHeureTypeSens.groupby(['date_heure','sens']).agg({'nbVeh':'sum','heure':lambda x : x.unique()[0], 
                    'jour':lambda x : x.unique()[0],'jourAnnee':lambda x : x.unique()[0],'date':lambda x : x.unique()[0]}
                    ).assign(type_veh='TV').reset_index(),dfHeureTypeSens], axis=0, sort=False).sort_values(['date_heure', 'type_veh'])

In [ ]:
dfGroupVtsHeure=pd.concat([dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure').assign(type_veh='TV'),
           dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure')])
dfHeureTypeSensV=dfHeureTypeSens.merge(dfGroupVtsHeure, on=['date_heure','sens', 'type_veh'],how='left').sort_values(['date_heure', 'type_veh', 'sens'])

In [36]:
dossier = r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\Brut'
bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 110, 9999]
cpt = ds.ComptageDonneesIndiv(dossier, vitesse=True)
cpt.dfSemaineMoyenne.groupby(['sens', 'type_veh']).agg(
            **{'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}).reset_index().to_csv(
    r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\SortiePluginQgis\P8_MJO.csv')

In [64]:
cpt.dicoNbJours

{'dfNbJours': 0    2
 1    2
 2    2
 3    1
 4    1
 5    1
 6    1
 Name: nbOcc, dtype: int64,
 'nbJours': 10,
 'nbJoursOuvres': 8}

In [53]:
test = cpt.df2SensBase.copy()
test['classe_vitesse'] = pd.cut(cpt.df2SensBase.vitesse, bins)#.value_counts()
test['jour'] = test.date_heure.dt.dayofyear
test['jourSemaine'] = test.date_heure.dt.dayofweek

In [63]:
groupe = test.groupby(['sens', 'type_veh', 'classe_vitesse', 'jourSemaine', 'jour']).nbVeh.sum().reset_index().sort_values(
    ['jour', 'classe_vitesse', 'type_veh', 'sens', 'jour'])
groupe.loc[groupe.jour==319]

,sens,type_veh,classe_vitesse,jourSemaine,jour,nbVeh
0,sens1,2R,"(0, 20]",0,319,1
10,sens1,2R,"(0, 20]",1,319,0
20,sens1,2R,"(0, 20]",2,319,0
30,sens1,2R,"(0, 20]",3,319,0
40,sens1,2R,"(0, 20]",4,319,0
50,sens1,2R,"(0, 20]",5,319,0
60,sens1,2R,"(0, 20]",6,319,0
2100,sens2,2R,"(0, 20]",0,319,0
2110,sens2,2R,"(0, 20]",1,319,0
2120,sens2,2R,"(0, 20]",2,319,0


In [59]:
test.loc[test.jour==319].jourSemaine.unique()

array([0], dtype=int64)

In [29]:
# initialisation et calcul des donnees
dossier = r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\Brut'
cpt = ds.ComptageDonneesIndiv(dossier, vitesse=True)

pd.concat([cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].groupby(['sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85)),
               'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}}).reset_index(),
cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].groupby(['type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85)),
               'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}}).reset_index().assign(sens='tout sens'),
cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].groupby(['sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85)),
               'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}).reset_index().assign(type_veh='TV'),
pd.DataFrame({'sens': ['tout sens'], 'type_veh': ['TV']}).assign(
    v50=cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].vitesse.quantile(),
    v10=cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].vitesse.quantile(0.1),
    v85=cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].vitesse.quantile(0.85),
    mjo=round(cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].nbVeh.sum()/5)).to_csv(
    r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\SortiePluginQgis\P8_indic_agrege_MJO.csv')

0195. 033.0008.  90.   1.  21.  11.  15.  17.  00.0350.1200.   2. N44.  49.  30. 138.W000.  34.  11. 460.
['sens1' 'sens2']


In [30]:
round(cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].nbVeh.sum()/5)

6080

In [ ]:
pd.concat([dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure').assign(type_veh='TV'),
           dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure')])

# affectation auto des points de comptage
> on part des points de comptages nouveau avec une géométrie (exemple dans le 47)<br>
***EN COURS<BR>CHAPITRE 6.2.2 ET 2CRITURE DE LA FONCTION creerComptageAssoc(). L'IDEE EST DE CREER LES DF DIRECTEMENT TELEVERSABLES DANS LA BDD.<br>ENSUITE IL FAUDRA VERIFIER QUE TOUTE LES ETAPES DU SCHEMA [geolocaliserPtComptage](C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\travail\methode\Schemas_OTV.mdj) SONT BIEN REALISEE. JE PENSE QUE LE PB A L'EXECUTION DE LA FONCTION VIENT DE LA FONCTIO MERGE QUI CREE DES _x ET _y SUR DES ATTRIBUTS QUE JE SOUHAITE REFERENCER ENSUITE***


In [2]:
# recupérer les données
# params : nom de table  ou df des points geolocalises, nom de table de linauto, nom de Bdd
with ct.ConnexionBdd('local_otv_boulot') as c:
    pt47 = gp.read_postgis('select * from cd47_2020', c.sqlAlchemyConn)
    pt47Geom = gp.read_postgis('select * from cd47_2020 where geom is not null', c.sqlAlchemyConn)
    cptSansGeom = gp.read_postgis('select * from cd47_2020 where geom is null', c.sqlAlchemyConn)
    linauto47 = gp.read_postgis("""
                                 select DISTINCT ON (t.gid) c.id_comptag id_comptag_bdd,c.type_poste type_poste_bdd, t.gid, t.geom, st_distance(c.geom, t.geom)
                                  from (SELECT * FROM linauto.traf2020_bdt_na_ed20_simpli_l where list_dept like '%%47%%') t LEFT JOIN comptage.compteur c ON st_dwithin(c.geom, t.geom, 30)
                                  ORDER BY t.gid, CASE WHEN c.type_poste = 'permanent' THEN 1 
                                                       WHEN c.type_poste = 'tournant' THEN 2
                                                       WHEN c.type_poste = 'ponctuel' THEN 3 
                                                       ELSE 4 
                                                       end
                                 """, c.sqlAlchemyConn)

## Prealable
> ici on va preparer les donnes et fonctions generiques

### preparation des donnees

In [3]:
# on en déduit le plus proche voisin entre les points et la linauto
ppV = O.plus_proche_voisin(pt47Geom, linauto47, 10, 'id_comptag', 'gid')
ppVSectHomo = ppV.loc[~ppV.gid.isna()].copy()
ppvHorsSectHomo = ppV.loc[ppV.gid.isna()].copy()  # PENSER A CREER LES COMPTEURS POUR CEUX LA !!!!!
# et on récupère l'id_comptag_existant et l'id de section homogene
ppVTot = pt47Geom.merge(ppVSectHomo, how='left', on='id_comptag').merge(linauto47, how='left', on='gid')

# trouver le nombre de comptage par section homogène (gid)
nbCptSectHomo = ppVTot.gid.value_counts()

# separer les differents cas a traiter
cptSimpleSectHomo = ppVTot.loc[ppVTot.gid.isin(nbCptSectHomo.loc[nbCptSectHomo == 1].index.tolist())].copy()
cptMultiSectHomo = ppVTot.loc[ppVTot.gid.isin(nbCptSectHomo.loc[nbCptSectHomo > 1].index.tolist())].copy()

### Fonctions génériques

In [4]:
def creerCompteur(cptRef, attrGeom, dep, reseau, gestionnai, concession, techno=None, src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None,
                  id_sect=None, fictif=False, en_service=True):
    """
    creation d'une df prete a etre integree dans la table des compteurs
    presque tout les chmaps 
    in : 
        cptRef : ou geodataframe avec un attribut de géométrie en SRID = 2154
        attrGeom : string : nom de l'attribut supportant la géométrie
        dep : departement sur 2 caractère ou pd.Series
        reseau : valeur depuis liste enumeree (cf bdd) ou pd.Series de ces valeurs
        gestionnai : valeur depuis liste enumeree (cf bdd) ou pd.Series de ces valeurs
        concession : boolean ou pd.Series de booleean
    """
    df = cptRef.copy()
    O.checkAttributsinDf(df, [attrGeom, 'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt'])
    df = df[[attrGeom, 'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt']].assign(
        dep=dep, reseau=reseau, gestionnai=gestionnai, concession=concession, techno=techno, src_geo=src_geo, obs_geo=obs_geo,
        obs_supl=obs_supl, id_cpt=id_cpt, id_sect=id_sect, fictif=fictif, en_service=en_service)
    df['x_l93'] = df[attrGeom].apply(lambda x: round(x.x,3))
    df['y_l93'] = df[attrGeom].apply(lambda x: round(x.y,3))
    df.drop(
        [c for c in df.columns if c not in ('dep', 'reseau', 'gestionnai', 'concession', attrGeom,
                                            'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt', 'reseau', 'dep',
                                            'reseau', 'gestionnai', 'concession', 'techno', 'src_geo', 'obs_geo', 'obs_supl', 'id_cpt', 'id_sect', 'fictif',
                                            'en_service')], axis=1, inplace=True)
    gdfCpt = gp.GeoDataFrame(df, geometry=attrGeom, crs=2154)
    gdfCpt = O.gp_changer_nom_geom(gdfCpt, 'geom')
    return gdfCpt

In [69]:
def rangBddComptageAssoc(id_comptag_ref_Bdd, bdd='local_otv_boulot'):
    """
    Récupérer dans la bdd, le rang max associé à un comptage de réference dans la table comptage_assoc.comptage
    """
    with ct.ConnexionBdd(bdd) as c:
        rqt = f"""SELECT COALESCE(max(cac.rang),0) rang 
                   FROM comptage_assoc.comptage cac JOIN comptage.comptage c ON c.id = cac.id_cptag_ref 
                   WHERE c.id_comptag='{id_comptag_ref_Bdd}'"""
        rang = pd.read_sql(rqt, c.sqlAlchemyConn).iloc[0]
    return rang

def creerComptageAssoc(df, id_comptag_ref_nom, annee, id_compteur_asso_nom, src=None, listIdCptExclu=None):
    """
    a partir d'une df, creer la df a injecter dans les tables compteur et comptage du schema comptage_assoc de la bdd
    in : 
        df : la df de base, doit contenir obligatoirement les attributs decrivant l'id_comptag_ref, le rang, le type_veh. si possible la periode, src, obs.
             si le comptage_assoc comprend une dimension géométrique, les attributs decrivant l'id_compteur_asso, le type_poste, la src_geo, la src_cpt, convention,
             et sens_cpt sont obligatoire. si possible ajouter route, pr, abs, techno, obs_geo, obs_supl
        id_comptag_ref_nom : string : nom de l'attribut supportant l'id_comptag du point de référence
        id_compteur_asso_nom : string : nom de l'attribut supportant l'id_comptag du point associé si composante geometrique (i.e l'id_comptag issu des donnees gestionnaire)
        annee : string annee sur 4 caractères
        listIdCptExclu : liste d'identifiant de comptage a ne pas conserver dans les resultats
    out : 
        dfIds : df contenant tous les points de la df en entree, avec les attributs issus de la Bdd en plus
        tableComptage : df au format de la table des comptage Associes. ne contient que les points 
    """
    def remplirObsSelonVacances(obsExistant, vacances):
        """
        si unn champs obs existe deja, on ajoute un phrase sur le fait que le point a été réalisé pendant les vacances, sinon on 
        ne met ue la menstion relatives aux vacances
        in : 
            obsexistant : strin ou valeur nulle
            vacances : boolean
        out :
            string
        """
        if not pd.isnull(obsExistant):
            if vacances:
                return f'{obsExistant} ; une partie des mesures sont réalisées pendant les vacances scolaires'
        else:
            if vacances:
                return 'une partie des mesures sont réalisées pendant les vacances scolaires'
        return None
    
    # mise en forme
    dfSource = df.copy()
    O.checkAttributsinDf(dfSource, ['type_veh', 'periode'])
    if listIdCptExclu:
        dfSource = dfSource.loc[~dfSource[id_compteur_asso_nom].isin(listIdCptExclu)].copy()
    dfSource.rename(columns={id_compteur_asso_nom: 'id_cpteur_asso'}, inplace=True)
    instanceCpt = it.Comptage('fake')
    corresIdComptagInterne = instanceCpt.recupererIdUniqComptage(dfSource[[id_comptag_ref_nom]].rename(columns={id_comptag_ref_nom: 'id_comptag'}), True)
    dfIds = dfSource.merge(corresIdComptagInterne, left_on=id_comptag_ref_nom, right_on='id_comptag', how='left').rename(columns={'id_comptag_uniq': 'id_comptag_ref'}).assign(annee=annee)
    dfIds['rang_bdd'] = dfIds[id_comptag_ref_nom].apply(lambda x: rangBddComptageAssoc(x))
    dfIds['rang_df'] = dfIds.groupby(id_comptag_ref_nom).cumcount()+1
    dfIds['rang'] = dfIds['rang_bdd']+dfIds['rang_df']

    tableComptage = dfIds[[c for c in pb.attrComptageAssoc if c in dfIds.columns]].copy()
    # ajouter ou modifier le champs observation pour les ponctuels ou tournant dont une partie de la periode est en vacances scolaire
    if 'obs' in tableComptage.columns:
        tableComptage['obs'] = tableComptage.apply(lambda x: remplirObsSelonVacances(x.obs, O.verifVacanceRange(x.periode)), axis=1)
    else:
        tableComptage['obs'] = tableComptage.periode.apply(lambda x: remplirObsSelonVacances(None, O.verifVacanceRange(x)))
    return dfIds, tableComptage


def creerCommpteurAssoc(df, nomAttrIdCpteurAsso, nomAttrGeom=None, nomAttrIdCpteurRef=None, listIdCptExclu=None):
    """
    a partir d'une df, creer la table des compteurdu schema comptage_assoc de la bdd. la table source doit contenir les attributs d'ientifiantde comptage,
    type_poste, src_geo, src_cpt, convention, sens_cpt. 
    pour plus de precision elle peut contenir geom, route, pr, abs, techno, obs_geo, obs_supl, id_cpt, id_sect, id_cpteur_ref. Cf Bdd pour plus de détail
    in : 
        df : dataframe des donnees sources
        nomAttrIdCpteurAsso : string : nom de l'attribut contenant les id_comptag mis en forme a partir du gestionnaire
        nomAttrIdCpteurRef : string : nom de l'attribut contenant les id_comptag de référence de la table compteur du schema comptapge de la bdd
        nomAttrGeom : string : nom de l'attribut qui supporte la géométrie
    out :
        dataframe au format bdd comptage_assoc.compteur
    """
    O.checkAttributsinDf(df, [nomAttrIdCpteurAsso, 'type_poste', 'src_geo', 'src_cpt', 'convention', 'sens_cpt'])
    dfSource = df.copy()
    if listIdCptExclu:
        dfSource = dfSource.loc[~dfSource[nomAttrIdCpteurAsso].isin(listIdCptExclu)].copy()
    dfSource.rename(columns={nomAttrIdCpteurAsso: 'id_cpteur_asso'}, inplace=True)
    if nomAttrIdCpteurRef:
        dfSource.rename(columns={nomAttrIdCpteurRef: 'id_cpteur_ref'}, inplace=True)
    if nomAttrGeom:
        dfSource = O.gp_changer_nom_geom(dfSource, 'geom')
    tableCompteur = dfSource[[c for c in dfSource.columns if c in pb.attrCompteurAssoc]]
    return tableCompteur

In [8]:
def creerCorrespComptag(df, nomAttrIdComptagGest, nomAttrIdComptagGti, listIdCptExclu):
    """
    creer une df a integrer dans la bdd a partir d'une df comprenant : un attribut id_comptag de la bdd et un attribut id_comptag issu des données gestionnaire.
    si la df commprend des type poste gestionnaire et bdd on peut ajouter le nom de ces attributs pour faire un test 
    in :
        df : df de base doit contenir un attribut id_comptag de la bdd et un attribut id_comptag issu des données gestionnaire
        nomAttrIdComptagGest : string : nom de l'attribut décrivant l'id_comptag issu des données gestionnaire
        nomAttrIdComptagGti : string : nom de l'attribut décrivant l'id_comptag de la bdd
        listIdCptExclu : list ou tuple : list des id_comptage à ne pas prendre en compte
    """
    return df.loc[~df[nomAttrIdComptagGest].isin(listIdCptExclu)][[nomAttrIdComptagGest, nomAttrIdComptagGti]].copy().rename(columns={nomAttrIdComptagGest: 'id_gest', nomAttrIdComptagGti: 'id_gti'})

## CAS 1 : 1 seul comptage sur une section homogène
on va séparer les cas où un id_comptag est déjà présent ou non

In [9]:
cptSimpleSectHomoNew = cptSimpleSectHomo.loc[cptSimpleSectHomo.id_comptag_bdd.isna()].copy()
cptSimpleSectHomoOld = cptSimpleSectHomo.loc[~cptSimpleSectHomo.id_comptag_bdd.isna()].copy()

## CAS 2 : plusieurs comptages sur une section homogène
Il va falloir déterminer quel est le comptage de référence (cf doc)

pour pouvoir déterminer le comptage de référence il faut connaitre : 
1. les types de postes (priorité : permanent > tournant > ponctuel)
1. les périodes de comptages (permanent non concerné)
1. les vacances scolaires
1. les niveaux de TMJA
1. le niveau de pc_pl
1. un choix arbitraire

### fonction de choix d'un comptage de reference

In [81]:
# creation de la fonction de hiérarchisation
def hierarchisationCompteur(typePoste, periode, tmja, pc_pl):
    """
    obtention d"'une note globale de hierarhisation, a partir des 3 fonction incluse'
    """
    def hierarchisationTypePoste(typePoste):
        """
        attribuer une valeur de 3*10**21, 2*10**21 ou 10**21 selon le type de poste
        in : 
            typePoste : string parmi 'permanent', 'tournant', 'ponctuel'
        """
        O.checkParamValues(typePoste, ('permanent', 'tournant', 'ponctuel'))
        if typePoste == 'permanent':
            return 3*10**15
        elif typePoste == 'tournant':
            return 2*10**15
        else:
            return 10**15

    def hierarchisationVacance(periode):
        """
        attribuer une valeur de 2*10**18 ou 10**18 selon qu'une partie de la mesure a été faie pendant les vacances scolaire
        in :
            periode : string sou forme YYYY/MM/DD-YYYY/MM/DD éventuellement séparé par des ' ; '
        """
        testVacances = O.verifVacanceRange(periode)
        if not periode or testVacances:
            return 10**12
        else:
            return 2*10**12

    def hierarchisationTrafic(tmja, pc_pl):
        """
        fournir une valeur qui concatene le trafic et le pc_pl, en favorisant le trafic
        """
        if not tmja or tmja <= 0: 
            return ValueError('le tmja doit etre une superieur a 0  non nulle')
        elif not pc_pl:
            return tmja * 1000
        else:
            return (tmja * 1000) + pc_pl

    return hierarchisationTypePoste(typePoste) + hierarchisationVacance(periode) + hierarchisationTrafic(tmja, pc_pl)

In [82]:
# calcul des compteur de reference et associes
cptMultiSectHomo['note_hierarchise'] = cptMultiSectHomo.apply(lambda x: hierarchisationCompteur(x.type_poste, x.periode, x.tmja, x.pc_pl), axis=1)
cptRefMultiSectHomo = cptMultiSectHomo.loc[cptMultiSectHomo.groupby('gid').note_hierarchise.transform('max') == cptMultiSectHomo.note_hierarchise].sort_values('gid').copy()
cptAssocMultiSectHomo = cptMultiSectHomo.loc[cptMultiSectHomo.groupby('gid').note_hierarchise.transform('max') != cptMultiSectHomo.note_hierarchise].sort_values('gid').copy()
cptAssocMultiSectHomo = cptAssocMultiSectHomo.merge(cptRefMultiSectHomo[['gid', 'id_comptag']], on='gid', suffixes=(None, '_ref'))
# verif que tous les gid ont un cpt ref
if not cptAssocMultiSectHomo.loc[~cptAssocMultiSectHomo.gid.isin(cptRefMultiSectHomo.gid.unique())].empty:
    raise ValueError('un des comptage associe n\'a pas de comptage de reference')
if not len(cptRefMultiSectHomo)+len(cptAssocMultiSectHomo) == len(cptMultiSectHomo):
    raise ValueError('un ou plusieurs comptage n\'ont pas ete affecte comme reference ou associe')

In [85]:
# on sépare les comptages de références selon qu'ils sont sur un id_comptag de la bdd ou non
cptRefMultiSectHomoNew = cptRefMultiSectHomo.loc[cptRefMultiSectHomo.id_comptag_bdd.isna()].copy()
cptRefMultiSectHomoOld = cptRefMultiSectHomo.loc[~cptRefMultiSectHomo.id_comptag_bdd.isna()].copy()

## Comptage de référence sur section_homogene sans id_comptag
creation des compteurs mais validation finale apres analyse des donnes sur section homogene avec id_comptag

In [87]:
# creation des compteurs
dfCompteurNewBruteAgrege = pd.concat([cptSimpleSectHomoNew, cptRefMultiSectHomoNew])

# A CE STADE ON NE CREE PAS ENCORE LES COMPTEURS ET COMPTAGES : ON VERIFIE D'ABORD QU'IL FAUT PAS EN AJOUTER CAR MAL CLASSES DANS D'AUTRE CATEGORIES

## Comptage de référence sur section_homogene avec id_comptag
selon la nature du comptage deja présent dans la base et selon la nature du comptage a injecter, on va soit : 
- creer un nouveau comptage associe avec le nouveau point
- creer une correspondance dans corresp_id_comptag
- modifier le type de poste et tracer la modif
- décider après analyse manuelle de creer un nouveau compteur

In [122]:
def ventilerNouveauComptageRef(df, nomAttrtypePosteGest, nomAttrtypePosteBdd, nomAttrperiode):
    """
    depuis une df des comptage de référence situé sur des tronçons avec un id_comptag existantdans la base, 
    séparer en 3 groupe selon le type de poste dans la bdd et le type de poste du hestionnaire.
    in : 
        df : dataframe a classifier
        nomAttrtypePosteGest : string : nom de l'attribut supportant le type de poste fourni par le gest 
        nomAttrtypePosteBdd : string : nom de l'attribut supportant le type de poste dans la bdd
        nomAttrperiode : string : nom de l'attribut décrivant la période de mesure
    """
    dfCorrespIdComptag = df.loc[((df[nomAttrtypePosteGest] == df[nomAttrtypePosteBdd]) &
                                 (df[nomAttrtypePosteGest] != 'ponctuel')) |
                                ((df[nomAttrtypePosteGest] == df[nomAttrtypePosteBdd]) &
                                 (df[nomAttrtypePosteGest] == 'ponctuel') &
                                 (~df[nomAttrperiode].apply(lambda x: O.verifVacanceRange(x))))].copy()
    dfCreationComptageAssocie = df.loc[((df[nomAttrtypePosteGest] == 'ponctuel') &
                                        (df[nomAttrtypePosteBdd].isin(('permanent', 'tournant')))) |
                                       ((df[nomAttrtypePosteGest] == df[nomAttrtypePosteBdd]) &
                                        (df[nomAttrtypePosteGest] == 'ponctuel') &
                                        (df[nomAttrperiode].apply(lambda x: O.verifVacanceRange(x))))].copy()
    dfModifTypePoste = df.loc[((df[nomAttrtypePosteGest].isin(('permanent', 'tournant'))) & (df[nomAttrtypePosteBdd] == 'ponctuel')) |
                              ((df[nomAttrtypePosteGest] == 'permanent') & (df[nomAttrtypePosteBdd] == 'tournant'))].copy()
    return dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste

In [123]:
dfCompteurExistantAVentiler = pd.concat([cptSimpleSectHomoOld, cptRefMultiSectHomoOld])

In [ ]:
# pensez à vérifier d'éventuel pb de doublons du à la linauto en bordure de dept
dfCompteurExistantAVentiler.loc[dfCompteurExistantAVentiler.duplicated('index', keep = False)]

In [130]:
# correction
dfCompteurExistantAVentiler.drop(34, inplace=True)

In [124]:
dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste = ventilerNouveauComptageRef(dfCompteurExistantAVentiler, 'type_poste', 'type_poste_bdd', 'periode')

In [ ]:
# analyse de la table de correspiIdComptag
# export pour vérif
gp.GeoDataFrame(dfCorrespIdComptag.drop('geom_y', axis=1), geometry='geom_x', crs=2154
                ).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifCorrespIdComptage.shp')

In [132]:
# certains comptages doivent etre "forcé" en nouveau compteur. On fait une liste
listCompteurAForcer = []
listCompteurAForcer.append('47-D211-8+0')
listCompteurAForcer.append('47-D276-10+905')

In [115]:
# analyse de la table de dfCreationComptageAssocie
# export pour vérif
gp.GeoDataFrame(dfCreationComptageAssocie.drop('geom_y', axis=1), geometry='geom_x', crs=2154
                ).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifComptagesAssoc.shp')

In [136]:
# on ajoute les valeurs dans la liste des compteurs a forcer
listCompteurAForcer.append('47-D124-0+315')

In [ ]:
# analyse des modifs de type de poste : -> null
dfModifTypePoste

In [ ]:
# RESUME une ligne par donnee 
# correspIdComptag
# tableComptageAssoc
# tableCompteurAssoc

## creation des nouveaux compteurs

### recuperation des compteurs forcés depuis les comptages associés

In [139]:
dfCompteurNewSupp = dfCompteurExistantAVentiler.loc[dfCompteurExistantAVentiler.id_comptag.isin(listCompteurAForcer)].copy()
dfCompteurNewBruteAgregeFinal = pd.concat([dfCompteurNewSupp, dfCompteurNewBruteAgrege])
attrGeomCompteurNew = 'geom_x'
dep = '47'
reseau = 'RD'
gestionnai = 'CD47'
concession = 'N'
dfCompteurNewBruteAgregeFinal['src_cpt'] = dfCompteurNewBruteAgregeFinal.type_poste.apply(lambda x: 'convention_gestionnaire' if x == 'permanent' else 'gestionnaire')
dfCompteurNewBruteAgregeFinal['convention'] = dfCompteurNewBruteAgregeFinal.type_poste.apply(lambda x: True if x == 'permanent' else False)
dfCompteurNewBruteAgregeFinal['sens_cpt'] = 'double sens'
dfCompteurNew = creerCompteur(dfCompteurNewBruteAgregeFinal, attrGeomCompteurNew, dep, reseau, gestionnai, concession, techno=None, 
                              src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None, id_sect=None, fictif=False, en_service=True)

In [139]:
# inserer dans la base

### creation des comptages

In [ ]:
# creation des comptages (il faut inserer les compteurs avant)
cptFictif = it.Comptage('faux')
cptFictif.creer_comptage(dfCompteurNewBruteAgrege.id_comptag.tolist(), 2020, 'donnees complete CD47', 'vl/pl', periode=dfCompteurNewBruteAgrege.periode.tolist())

### creation des indicateurs

In [ ]:
# creation des indicateurs (necessite la creation des comptages avant)
cptFictif.structureBddOld2NewForm(dfCompteurNewBruteAgrege.assign(annee='2020'), '2020', ['id_comptag','annee', 'fichier'], ['tmja', 'pc_pl'], 'agrege')

## creation des correspondances d'id_comptages

In [141]:
correspIdComptag = creerCorrespComptag(dfCorrespIdComptag, 'id_comptag', 'id_comptag_bdd', listCompteurAForcer)

In [142]:
# A INSERER EN BASE AVANT CREATION DES DONNEES 
correspIdComptag

,id_gest,id_gti
5,47-D8-9+832,47-D8-7+0
6,47-D8-9+953,47-D8-14+0
37,47-D211-1+0,47-D211-2+140
49,47-D147-4+0,47-D147-4+100
50,47-D259-5+750,47-D259-6+0
52,47-D156-0+70,47-D156-3+0
63,47-D103-0+400,47-D103-0+560
75,47-D442-1+850,47-D442-2+0
72,47-D225-5+80,47-D225-4+516
58,47-D414-0+933,47-D414-0+139


## creation des comptages associés (avec tables compteurs et indics)

In [143]:
# creer les comptages associés
dfIds, tableComptageAssoc = creerComptageAssoc(dfCreationComptageAssocie.assign(type_veh='vl/pl',
                                                           src_geo='pr+abs_gestionnaire',
                                                           convention=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: True if x == 'permanent' else False),
                                                           sens_cpt='double sens',
                                                           src_cpt=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: 'convention gestionnaire'
                                                                                                                        if x == 'permanent' else 'gestionnaire')),
                                                           'id_comptag_bdd', '2020', id_compteur_asso_nom='id_comptag', src=None, listIdCptExclu=listCompteurAForcer)

In [144]:
# creer les compteurs relatifs aux comptages associes
tableCompteurAssoc = creerCommpteurAssoc(gp.GeoDataFrame(dfIds, geometry='geom_x', crs=2154).rename(
    columns={'absc': 'abs'}), 'id_cpteur_asso', 'geom_x', 'id_comptag_ref', listCompteurAForcer)

In [60]:
# creer les indicateurs relatifs aux comptages agreges
# en fait il s'agit surtout de créer le lien entre l'id_comptag et l'id_cptag_uniq, sur le modele du schema comptage

In [146]:
tableComptageAssoc, tableCompteurAssoc

,id_comptag_ref,rang,periode,type_veh,src,id_cpteur_asso,obs
0,40683,1,2020/06/04-2020/06/10,vl/pl,donnees_xls_sources,47-D15-19+505,None
1,41014,1,2020/10/27-2020/11/02,vl/pl,donnees_xls_sources,47-D292-1+960,une partie des mesures sont réalisées pendant ...
2,40559,1,2020/09/09-2020/09/15,vl/pl,donnees_xls_sources,47-D129-2+362,None
3,41068,1,2020/09/09-2020/09/13,vl/pl,donnees_xls_sources,47-D308-2+578,None
4,41548,1,2020/06/26-2020/07/01,vl/pl,donnees_xls_sources,47-D813-31+725,None
5,41021,1,2020/10/27-2020/11/02,vl/pl,donnees_xls_sources,47-D292-2+140,une partie des mesures sont réalisées pendant ...
6,40325,1,2020/09/24-2020/09/30,vl/pl,donnees_xls_sources,47-D108-4+506,None
7,40370,1,2020/02/18-2020/02/24,vl/pl,donnees_xls_sources,47-D113-6+480,une partie des mesures sont réalisées pendant ...
8,40380,1,2020/02/18-2020/02/24,vl/pl,donnees_xls_sources,47-D113-6+890,une partie des mesures sont réalisées pendant ...
9,40418,1,2020/02/18-2020/02/24,vl/pl,donnees_xls_sources,47-D118-22+330,une partie des mesures sont réalisées pendant ...


# transfert des donnees raw TIPI dans la BDD
> on part du fichier transmis par l'equipe de Pascal GLASSON (C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\DIRA\historique\2020\hors_convention\tipi_alienor_raw.v2.zip) 


In [3]:
with ct.ConnexionBdd('local_otv_boulot') as c:
    ct.ogr2ogr_csv2pg(c.connstringOgr, r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', table='dira_tipi_2020_raw')

debut import fichier C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv avec shape2pg à 13:55:48 
 avec commande ogr2ogr -f "postgreSQL" --config PG_USE_COPY YES -lco "SCHEMA=public" PG:"host=localhost dbname=otv user=postgres password=postgres port=5432" C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv -nln public.dira_tipi_2020_raw -oo HEADERS=YES
Fait
